# 实现简单的自动梯度系统

## 📚 代码示例关联

**本示例位置**：这是自动梯度与优化的核心代码示例

**基于**：
- [从零实现前馈神经网络](../02_前馈神经网络/代码示例/01_从零实现前馈神经网络.ipynb) - 理解手动反向传播
- [从零实现神经元](../01_感知器与神经元/代码示例/01_从零实现神经元.ipynb) - 理解基本计算

**扩展为**：
- [PyTorch基础入门](../../04_PyTorch_TensorFlow/01_PyTorch基础/代码示例/01_PyTorch基础入门.ipynb) - PyTorch的autograd系统
- [TensorFlow基础入门](../../04_PyTorch_TensorFlow/02_TensorFlow基础/代码示例/) - TensorFlow的计算图
- [从零实现优化算法](../../05_网络优化与正则化/01_网络优化/代码示例/01_从零实现优化算法.ipynb) - 使用自动梯度实现优化算法

**相关示例**：
- [理解计算图](../练习题/基础练习/) - 基础练习
- [优化算法对比](../实战案例/) - 实战应用

---

## 学习目标

本notebook将带你从零开始实现一个简单的自动梯度系统，类似PyTorch的autograd，包括：

1. **理解计算图的概念**
   - 如何构建计算图
   - 如何存储计算历史

2. **理解自动梯度的原理**
   - 前向传播构建计算图
   - 反向传播计算梯度

3. **能够实现简单的自动梯度系统**
   - Variable类（存储数据和梯度）
   - Function类（定义操作）
   - 自动反向传播

## 课程概述

本示例将实现一个简化版的自动梯度系统，支持基本的数学运算（加法、乘法、激活函数等），并能够自动计算梯度。


In [ ]:
# 导入必要的库
import numpy as np
import matplotlib.pyplot as plt

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

# 设置随机种子
np.random.seed(42)

print("环境准备完成！")


## 1. Variable类实现

Variable类用于存储数据和梯度，并记录计算历史（用于反向传播）。


In [ ]:
class Variable:
    """
    自动梯度系统的核心类
    存储数据、梯度，并记录计算历史
    """
    
    def __init__(self, data, requires_grad=False):
        """
        初始化Variable
        
        参数:
        - data: 数据（numpy数组）
        - requires_grad: 是否需要计算梯度
        """
        self.data = np.array(data)
        self.requires_grad = requires_grad
        self.grad = None  # 梯度
        self.creator = None  # 创建此Variable的函数（用于反向传播）
        self.generation = 0  # 计算图的世代（用于确定反向传播顺序）
    
    def backward(self, grad=None):
        """
        反向传播
        
        参数:
        - grad: 从输出层传来的梯度（默认为1，表示标量输出）
        """
        if not self.requires_grad:
            return
        
        if grad is None:
            grad = np.ones_like(self.data)
        
        # 累加梯度（如果此Variable被多个操作使用）
        if self.grad is None:
            self.grad = grad
        else:
            self.grad = self.grad + grad
        
        # 如果有创建者，继续反向传播
        if self.creator is not None:
            self.creator.backward(self.grad)
    
    def __add__(self, other):
        """加法运算"""
        return Add()(self, as_variable(other))
    
    def __mul__(self, other):
        """乘法运算"""
        return Mul()(self, as_variable(other))
    
    def __pow__(self, power):
        """幂运算"""
        return Pow(power)(self)
    
    def __repr__(self):
        return f"Variable(data={self.data}, grad={self.grad}, requires_grad={self.requires_grad})"

def as_variable(obj):
    """将对象转换为Variable"""
    if isinstance(obj, Variable):
        return obj
    return Variable(obj)

print("Variable类定义完成！")


## 2. Function类实现

Function类定义操作的前向传播和反向传播。


In [ ]:
class Function:
    """
    所有操作的基类
    定义前向传播和反向传播
    """
    
    def __call__(self, *inputs):
        """
        调用函数（前向传播）
        
        参数:
        - inputs: 输入Variable列表
        
        返回:
        - outputs: 输出Variable列表
        """
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        
        # 如果输出不是元组，转换为元组
        if not isinstance(ys, tuple):
            ys = (ys,)
        
        # 创建输出Variable
        outputs = [Variable(y) for y in ys]
        
        # 设置计算图连接
        generation = max([x.generation for x in inputs], default=0)
        for output in outputs:
            output.creator = self
            output.generation = generation + 1
            output.requires_grad = any(x.requires_grad for x in inputs)
        
        self.inputs = inputs
        self.outputs = outputs
        
        return outputs[0] if len(outputs) == 1 else outputs
    
    def forward(self, *xs):
        """前向传播（子类必须实现）"""
        raise NotImplementedError
    
    def backward(self, gy):
        """反向传播（子类必须实现）"""
        raise NotImplementedError

class Add(Function):
    """加法操作"""
    
    def forward(self, x0, x1):
        return x0 + x1
    
    def backward(self, gy):
        """加法反向传播：梯度直接传递"""
        return gy, gy

class Mul(Function):
    """乘法操作"""
    
    def forward(self, x0, x1):
        return x0 * x1
    
    def backward(self, gy):
        """乘法反向传播：使用链式法则"""
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        return gy * x1, gy * x0

class Pow(Function):
    """幂运算"""
    
    def __init__(self, power):
        self.power = power
    
    def forward(self, x):
        return x ** self.power
    
    def backward(self, gy):
        """幂运算反向传播"""
        x = self.inputs[0].data
        return gy * self.power * (x ** (self.power - 1))

class Sigmoid(Function):
    """Sigmoid激活函数"""
    
    def forward(self, x):
        x = np.clip(x, -500, 500)  # 防止溢出
        return 1 / (1 + np.exp(-x))
    
    def backward(self, gy):
        """Sigmoid反向传播"""
        y = self.outputs[0].data
        return gy * y * (1 - y)

class ReLU(Function):
    """ReLU激活函数"""
    
    def forward(self, x):
        return np.maximum(0, x)
    
    def backward(self, gy):
        """ReLU反向传播"""
        x = self.inputs[0].data
        return gy * (x > 0).astype(float)

print("Function类定义完成！")


In [ ]:
# 测试1：简单的加法和乘法
print("=" * 50)
print("测试1：y = (x1 + x2) * x3")
print("=" * 50)

x1 = Variable(np.array(2.0), requires_grad=True)
x2 = Variable(np.array(3.0), requires_grad=True)
x3 = Variable(np.array(4.0), requires_grad=True)

# 前向传播
y = (x1 + x2) * x3

print(f"x1 = {x1.data}")
print(f"x2 = {x2.data}")
print(f"x3 = {x3.data}")
print(f"y = {y.data}")
print()

# 反向传播
y.backward()

print("梯度:")
print(f"∂y/∂x1 = {x1.grad} (应该是 x3 = {x3.data})")
print(f"∂y/∂x2 = {x2.grad} (应该是 x3 = {x3.data})")
print(f"∂y/∂x3 = {x3.grad} (应该是 x1+x2 = {x1.data + x2.data})")
print()

# 验证：手动计算
# y = (x1 + x2) * x3
# ∂y/∂x1 = x3 = 4
# ∂y/∂x2 = x3 = 4
# ∂y/∂x3 = x1 + x2 = 5
assert np.allclose(x1.grad, 4.0), f"x1.grad应该是4.0，得到{x1.grad}"
assert np.allclose(x2.grad, 4.0), f"x2.grad应该是4.0，得到{x2.grad}"
assert np.allclose(x3.grad, 5.0), f"x3.grad应该是5.0，得到{x3.grad}"
print("✅ 测试1通过！")


## 4. 测试激活函数

测试Sigmoid和ReLU激活函数的自动梯度计算。


In [ ]:
# 测试2：Sigmoid激活函数
print("=" * 50)
print("测试2：y = sigmoid(x)")
print("=" * 50)

x = Variable(np.array(0.0), requires_grad=True)
y = Sigmoid()(x)

print(f"x = {x.data}")
print(f"y = sigmoid(x) = {y.data:.4f}")
print()

# 反向传播
y.backward()

# 手动计算验证
# sigmoid'(x) = sigmoid(x) * (1 - sigmoid(x))
expected_grad = y.data * (1 - y.data)

print(f"∂y/∂x = {x.grad:.4f}")
print(f"期望值 = {expected_grad:.4f}")
assert np.allclose(x.grad, expected_grad), "Sigmoid梯度计算错误"
print("✅ 测试2通过！")
print()

# 测试3：ReLU激活函数
print("=" * 50)
print("测试3：y = ReLU(x)")
print("=" * 50)

# 正数情况
x1 = Variable(np.array(2.0), requires_grad=True)
y1 = ReLU()(x1)
y1.backward()
print(f"x = {x1.data}, y = ReLU(x) = {y1.data}, ∂y/∂x = {x1.grad} (应该是1)")

# 负数情况
x2 = Variable(np.array(-2.0), requires_grad=True)
y2 = ReLU()(x2)
y2.backward()
print(f"x = {x2.data}, y = ReLU(x) = {y2.data}, ∂y/∂x = {x2.grad} (应该是0)")

assert np.allclose(x1.grad, 1.0), "ReLU正数梯度应该是1"
assert np.allclose(x2.grad, 0.0), "ReLU负数梯度应该是0"
print("✅ 测试3通过！")


## 5. 复杂计算图测试

测试更复杂的计算图，类似神经网络的单层。


In [ ]:
# 测试4：复杂计算图 y = sigmoid(w * x + b)
print("=" * 50)
print("测试4：y = sigmoid(w * x + b)")
print("=" * 50)

w = Variable(np.array(0.5), requires_grad=True)
x = Variable(np.array(2.0), requires_grad=True)
b = Variable(np.array(0.3), requires_grad=True)

# 前向传播
z = w * x + b
y = Sigmoid()(z)

print(f"w = {w.data}")
print(f"x = {x.data}")
print(f"b = {b.data}")
print(f"z = w * x + b = {z.data}")
print(f"y = sigmoid(z) = {y.data:.4f}")
print()

# 反向传播
y.backward()

print("梯度:")
print(f"∂y/∂w = {w.grad:.6f}")
print(f"∂y/∂x = {x.grad:.6f}")
print(f"∂y/∂b = {b.grad:.6f}")
print()

# 手动验证
# y = sigmoid(w*x + b)
# ∂y/∂z = y * (1 - y)
# ∂y/∂w = ∂y/∂z * ∂z/∂w = y*(1-y) * x
# ∂y/∂x = ∂y/∂z * ∂z/∂x = y*(1-y) * w
# ∂y/∂b = ∂y/∂z * ∂z/∂b = y*(1-y) * 1

dz_dy = y.data * (1 - y.data)
expected_w_grad = dz_dy * x.data
expected_x_grad = dz_dy * w.data
expected_b_grad = dz_dy

print("手动计算验证:")
print(f"∂y/∂w 期望值 = {expected_w_grad:.6f}")
print(f"∂y/∂x 期望值 = {expected_x_grad:.6f}")
print(f"∂y/∂b 期望值 = {expected_b_grad:.6f}")

assert np.allclose(w.grad, expected_w_grad), "w梯度计算错误"
assert np.allclose(x.grad, expected_x_grad), "x梯度计算错误"
assert np.allclose(b.grad, expected_b_grad), "b梯度计算错误"
print("✅ 测试4通过！")


## 6. 使用自动梯度训练简单模型

使用我们的自动梯度系统训练一个简单的线性模型。


In [ ]:
# 生成简单的线性数据
np.random.seed(42)
X = np.random.randn(100, 1)
y_true = 2 * X + 1 + 0.1 * np.random.randn(100, 1)

# 初始化参数
w = Variable(np.random.randn(1, 1), requires_grad=True)
b = Variable(np.zeros(1), requires_grad=True)

# 训练参数
learning_rate = 0.01
epochs = 100

losses = []

print("开始训练...")
for epoch in range(epochs):
    total_loss = 0
    
    # 遍历所有样本（批量梯度下降）
    for i in range(len(X)):
        # 前向传播
        x = Variable(X[i:i+1], requires_grad=False)
        y_pred = w * x + b
        
        # 计算损失（均方误差）
        loss = (y_pred - y_true[i]) ** 2
        
        # 反向传播
        loss.backward()
        
        # 更新参数
        w.data -= learning_rate * w.grad
        b.data -= learning_rate * b.grad
        
        # 清零梯度
        w.grad = None
        b.grad = None
        
        total_loss += loss.data
    
    avg_loss = total_loss / len(X)
    losses.append(avg_loss)
    
    if (epoch + 1) % 20 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}, w: {w.data[0,0]:.4f}, b: {b.data[0]:.4f}")

print(f"\n训练完成！")
print(f"真实参数: w=2.0, b=1.0")
print(f"学习到的参数: w={w.data[0,0]:.4f}, b={b.data[0]:.4f}")

# 绘制损失曲线
plt.figure(figsize=(8, 5))
plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('训练损失曲线')
plt.grid(True, alpha=0.3)
plt.show()


## 7. 总结

### 7.1 关键要点

1. **计算图**：
   - 通过Variable和Function自动构建
   - 记录计算历史，用于反向传播

2. **自动梯度**：
   - 前向传播：构建计算图，计算函数值
   - 反向传播：使用链式法则计算梯度

3. **实现要点**：
   - Variable存储数据和梯度
   - Function定义操作的前向和反向
   - 通过creator连接计算图

### 7.2 与PyTorch的对比

我们的实现是PyTorch autograd的简化版：
- **相似点**：Variable、Function、自动反向传播
- **简化点**：不支持更多操作、不支持GPU、不支持动态图优化

### 7.3 进一步学习

- [PyTorch基础入门](../../04_PyTorch_TensorFlow/01_PyTorch基础/代码示例/01_PyTorch基础入门.ipynb) - 学习PyTorch的autograd系统
- [TensorFlow基础](../../04_PyTorch_TensorFlow/02_TensorFlow基础/) - 学习TensorFlow的计算图
- [从零实现优化算法](../../05_网络优化与正则化/01_网络优化/代码示例/01_从零实现优化算法.ipynb) - 使用自动梯度实现优化算法
